In [2]:
import pandas as pd
import requests
import math
import time
from tqdm import tqdm
import os
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import xmltodict as saveme
from tqdm import tqdm
import timeit
#pip install xmltodict



# Get the working and data directory paths
wkdir = os.path.dirname(os.getcwd())
data_dir  = wkdir + '/data'

#Load in the SCOPUS WOS Search Results
search_results = pd.read_csv(data_dir+'/scopus_search_results_r4.csv', low_memory = False)

# !! Import list of API Keys
api_keys = open(wkdir+'/code/keys.gitignore', "r").read().splitlines()


print('Master dataframe of search results imported with '+str(len(search_results['eid'].unique()))+' unique entries.')

#split list into 19 chunks:

chunks = range(8713, int(8e4)+8713, 2)

print('Beginning of song to retrieve abstracts:')

for i, chunk in enumerate(chunks):
    print('Retrieving abstracts ' +str(i+8713)+ ' to ' +str(str(i+int(1e4)+8713)))
    #set the api_key for this chunk
    api_key = api_keys[i]
    
    #make an empty list to store failures
    failed_pages = pd.DataFrame()

    # create master dataframe with all the first article in the list

    idx = search_results.iloc[chunk]['dc:identifier'].split(':')[1]
    response = requests.get('https://api.elsevier.com/content/abstract/scopus_id/'+idx+'?view=META&apikey='+ api_key)

    abstracts = pd.json_normalize(saveme.parse(response.text))

    for j, a in tqdm(search_results[chunk+1:chunk+int(1e4)].iterrows(), desc='Chunk '+str(i)):
        try:
            #print("Working on article " + str(i) +' of'  + str(len(search_results)))
            #Get scupis ID:
            idx = a['dc:identifier'].split(':')[1]

            #Format URL to retrieve the abstract:
            response = requests.get('https://api.elsevier.com/content/abstract/scopus_id/'+idx+'?view=META_ABS&apikey='+ api_key)
            abstract = pd.json_normalize(saveme.parse(response.text))
            abstracts = abstracts.append(abstract)
            
            time.sleep(0.12)
            
        except:
            failed_pages[j] = idx
            continue
            
    abstracts.to_csv(wkdir+'/data/abstracts_06192020_chunk_'+str(i+11)+'.csv')
    
    if len(failed_pages) > 0:
        failed_pages.to_csv(wkdir+'/data/failed_queries_chunk'+str(i+11)+'.csv')
        print('Failed abstract retrievals (' + str(len(failed_pages))+') saved to '+ data_dir)
    else:
        print('Zero failed retrievals.')
        continue
    
    time.sleep(0.12)
        
    print('Done with chunk '+str(i+12))

print("Ding, end of song!")

Master dataframe of search results imported with 141901 unique entries.
Beginning of song to retrieve abstracts:
Retrieving abstracts 0 to 10000


AttributeError: module 'pandas' has no attribute 'json_normalize'

In [ ]:
## Check results that we already have



In [ ]:
# cali = pd.DataFrame({'dates': dates, 'calis': cals})
# cali = cali.set_index('dates')

# cali.sum()

In [384]:
# grps = pd.DataFrame(search_results.eid.groupby(search_results.index.year).count())

# grps['calis'] = yrs['calis']

# ax = grps.plot(x=grps.index, kind='bar')

# # Make most of the ticklabels empty so the labels don't get too crowded
# ticklabels = ['']*len(search_results.index)
# # Every 4th ticklable shows the month and day
# #ticklabels[::4] = [item.strftime('%b %d') for item in search_results.index[::4]]
# # Every 12th ticklabel includes the year
# ticklabels[::12] = [item.strftime('%b %d\n%Y') for item in search_results.index[::12]]
# ax.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels))
# ax.set_ylabel('article count')
# ax.set_title('"ground water" OR groundwater OR ground-water OR aquifer')
# plt.tight_layout()
# plt.savefig('article_counts.png', dpi=300)